<a href="https://colab.research.google.com/github/defdc/Hoax_Detection_using_SVM/blob/main/Hoax_Detection_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# Walk through the directory and print the path of each file
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Install Sastrawi for Indonesian stemming
!pip install Sastrawi

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import nltk  # Natural Language Toolkit for text processing
from nltk.tokenize import word_tokenize  # Tokenize text into words
from sklearn.preprocessing import LabelEncoder  # Encode labels with value between 0 and n_classes-1
from sklearn.feature_extraction.text import TfidfVectorizer  # Convert text to TF-IDF features
from sklearn import svm  # Support Vector Machine classifier
from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier
from sklearn.metrics import accuracy_score  # Measure accuracy of the model
from sklearn.model_selection import train_test_split  # Split data into train and test sets
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Create stemmer for Indonesian language

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Download 'punkt' tokenizer models
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Stemmer**

Kita dapat menggunakan Stemmer untuk mengurangi jumlah word yang ada pada dataset

Contoh Hasil Stem
Perekonomian -> ekonomi pertumbuhan -> tumbuh

Tf-idf (Term Frequency-Inverse document frequency)
Term Frequency(tf) adalah jumlah kemunculan term

In [ ]:
# Create a stemmer object
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Initialize label encoder
Encoder = LabelEncoder()

# Initialize TF-IDF vectorizer
Tfidf_vect = TfidfVectorizer()

**Import Dataset**

**Feature**

ID - ID Data

label - 1 for False news, 0 for True fact

tanggal - tanggal narasi ditemukan

judul - judul/tagline berita yang tersebar

narasi - isi berita

nama file gambar - file gambar (tidak digunakan pada tutorial ini)

In [ ]:
data_train = pd.read_csv('https://raw.githubusercontent.com/ghazimuharam/nlp-hoax-recognition/master/Data%20Latih/Data%20Latih%20BDC.csv')

In [ ]:
# Display the first few rows of the dataset
data_train.head()

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [ ]:
# Display the count of each label
data_train['label'].value_counts()

label
1    3465
0     766
Name: count, dtype: int64

**Data Preprocessing**

In [ ]:
false_news = data_train[data_train['label'] == 1].sample(frac=1)
true_fact = data_train[data_train['label'] == 0]
df = pd.concat([true_fact, false_news.head(len(true_fact) + 200)])

df

,ID,label,tanggal,judul,narasi,nama file gambar
19,6775,0,25-Oct-17,Sandi Sarankan Warga Ngadu ke Kelurahan Saja,Astaga! Sandiaga Sarankan Warga Ngadu ke Kelur...,6775.jpg
21,7038,0,13-Jul-17,"Irina, Istri Ahli IT Hermansyah disebut Mantan...","Irina, Istri Ahli IT Hermansyah disebut Mantan...",7038.jpg
22,7214,0,8-Mar-17,"Nezar Patria adalah Kader PKI, Ikut Rapat Seti...","Nezar Patria adalah Kader PKI, Ikut Rapat Seti...",7214.jpg
55,13483,0,17-Jul-18,Coach Indra Sjafri: Saya Pernah 1 Tahun Tak Di...,Kendati sukses mengantarkan Timnas Indonesia U...,13483.png
58,13907,0,28-Apr-18,Kronologis Polisi Berondong Sedan Terobos Razi...,"Innalillah… Hanya Karena Lalai Terobos Razia, ...",13907.png
...,...,...,...,...,...,...
1497,344135,1,17-Aug-20,Akun Facebook Wali Kota Payakumbuh Riza Falepi,Akun Facebook Wali Kota Payakumbuh “Riza Falepi”,344135.jpg
3109,734767,1,5-May-18,Harusnya Pak Fadli Zon Sadar,"Harusnya Pak Fadli Zon Sadar. Kami Paspampres,...",734767.jpg
3168,748029,1,19-Oct-19,"Ini Baru yang namanya Aksi Teroris Biadab, Kej...",Mengerikan Ini Baru yang namanya Aksi Teroris ...,748029.png
257,58725,1,13-Apr-20,Pesan Berantai PT MOSI Tawarkan Rapid Test Kit...,"Dear bapak/ibu dalam masa pandemic ini, kami P...",58725.jpg



Feature Engineering

In [ ]:
feature = df['narasi']
label = df['label']

In [ ]:
# Convert text to lowercase and apply stemming
lower = [stemmer.stem(row.lower()) for row in feature]

# Display the first few processed texts
lower[:5]

# Tokenize the processed texts
tokens = [word_tokenize(element) for element in lower]

# Display the first few tokenized texts
tokens[:5]

[['astaga',
  'sandiaga',
  'saran',
  'warga',
  'ngadu',
  'ke',
  'lurah',
  'saja',
  'tak',
  'perlu',
  'ke',
  'balai',
  'kota'],
 ['irina', 'istri', 'ahli', 'it', 'hermansyah', 'sebut', 'mantan', 'psk'],
 ['nezar',
  'patria',
  'adalah',
  'kader',
  'pki',
  'ikut',
  'rapat',
  'tiap',
  'malam',
  'di',
  'istana',
  'telah',
  'jam',
  '8',
  'malam',
  'demikian',
  'tuduh',
  'dari',
  'alfian',
  'tandjung'],
 ['kendati',
  'sukses',
  'antar',
  'timnas',
  'indonesia',
  'u-19',
  'juara',
  'piala',
  'aff',
  'u-19',
  '2013',
  'dan',
  'lolos',
  'ke',
  'putar',
  'final',
  'piala',
  'asia',
  'u-19',
  '2014'],
 ['innalillah',
  'hanya',
  'karena',
  'lalai',
  'terobos',
  'razia',
  'mobil',
  'isi',
  'satu',
  'keluarga',
  'tembak',
  'polisi',
  'satu',
  'orang',
  'tewas']]

In [ ]:
tokens = [word_tokenize(element) for element in lower]

tokens[:1]

[['astaga',
  'sandiaga',
  'saran',
  'warga',
  'ngadu',
  'ke',
  'lurah',
  'saja',
  'tak',
  'perlu',
  'ke',
  'balai',
  'kota']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tokens, label, test_size=0.3, stratify=label)

In [ ]:
print('X_train : ', len(X_train))
print('X_test : ', len(X_test))

X_train :  1212
X_test :  520


In [ ]:
df = pd.DataFrame(X_test)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,draf,revisi,uu,ketenagakerjaan,hapus,cuti,haid,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,undang,liput,yang,sebar,dengan,atas,nama,ratna,sarumpaet,crisis,...,None,None,None,None,None,None,None,None,None,None
2,yth,seluruh,anggota,grup,sesuai,instruksi,gubernur,dki,jakarta,hasil,...,None,None,None,None,None,None,None,None,None,None
3,sereem,iiiiih,beras,dari,kantong,plastik,jahat,bnget,nieeh,orng,...,None,None,None,None,None,None,None,None,None,None
4,jakarta,barat,ada,yang,nyata,tak,aman,dengan,edar,voice,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

y_train

array([1, 0, 1, ..., 1, 0, 1])

In [ ]:
Tfidf_vect.fit(["".join(row) for row in X_train])

TfidfVectorizer()

In [ ]:
X_train_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_train])
X_test_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_test])

Model Training & Prediction

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=1, gamma="auto", verbose=True)
SVM.fit(X_train_Tfidf, y_train)

predictions_SVM = SVM.predict(X_test_Tfidf)
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, y_test)*100)

[LibSVM]SVM Accuracy Score ->  56.15384615384615
